In [ ]:
# Mount driver colab

path_mount = '/content/drive/'

from google.colab import drive
drive.mount(path_mount)

In [ ]:
from typing import Any, List, Tuple
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc 

In [ ]:
# Environment Variable

path_base = '/content/drive/My Drive/Classroom/PDI/Projeto_AB2.2'
path_imgs = 'segimage-pdi/imgs'

In [ ]:
# Load directory

os.chdir(path_base)
os.listdir()

In [ ]:
# Repositório do Projeto: <https://github.com/jytaloramon/segimage-pdi.git>
# Executar para baixar as imagens de teste

!git clone https://github.com/jytaloramon/segimage-pdi.git

### Declarações

In [ ]:
# Classe de Segmentação

class SegImageGrowing:

    def __init__(self, image: List[List[int]]) -> None:

      self._image = np.array(image, np.float32)
      self._shape = self._image.shape
      self._image_avg_global = np.average(image)

    def run_segmentation(self,
                         seed_start_posi: Tuple[int, int],
                         seed_start_color: int,
                         target: int
    ) -> Tuple[Any,  List[Tuple[int, int, int]]]:

      img_out = np.zeros(self._shape, dtype=np.float32)

      queue: List[Tuple[int, int, int, int]] = [
          (seed_start_posi[0], seed_start_posi[1], seed_start_color, self._image[1][1])]

      neighborhood: List[Tuple[int, int]] = [(-1, -1), (-1, 0), (-1, 1), (1, -1),
                                              (1, 0), (1, 1), (0, -1), (0, 1)]

      event_queue: List[Tuple[int, int, int]] = [
          (seed_start_posi[0], seed_start_posi[1], seed_start_color)]

      img_out[seed_start_posi[0]][seed_start_posi[1]] = seed_start_color
      while len(queue) > 0:
        l, c, i, b = queue.pop(0)
        
        rep_out = -1
        
        for l_add, c_add in neighborhood:
          next_l, next_c = l + l_add, c + c_add

          if self._is_valid_pixel(next_l, next_c) and img_out[next_l][next_c] == 0:
            if abs(b - self._image[next_l][next_c]) <= target:
              img_out[next_l][next_c] = i
              queue.append((next_l, next_c, i, b))
              event_queue.append((next_l, next_c, i))
            else:
              new_i = 255 if i == 1 else 1
              img_out[next_l][next_c] = new_i

              if rep_out == -1:
                rep_out = self._image[next_l][next_c]

              queue.append(
                  (next_l, next_c, new_i, rep_out))
              event_queue.append((next_l, next_c, i))

      return img_out, event_queue

    def _is_valid_pixel(self, l: int, c: int) -> bool:

        return not(l < 0 or c < 0 or l >= self._shape[0] or c >= self._shape[1])

In [ ]:
def main_seg_image(name:str,  
                   init_pixel:Tuple[int, int],
                   v_init:int,
                   target:int
)->None:

  img_original = cv2.imread(f'{path_imgs}/{name}')
  img_original_gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
  
  img = img_original_gray

  seg_growing = SegImageGrowing(img)

  img_seg, events_seg = seg_growing.run_segmentation(init_pixel, v_init, target)

  print('- (1) Imagem Original (Colorida).')
  print('- (2) Imagem Original em Escala de Cinza (imagem usada)')
  print('- (3) Segmentação', end='\n\n')

  figsize = (15, 15)
  fig = plt.figure(figsize=figsize)
  fig.add_subplot(311).imshow(img_original, vmin=0, vmax=255)
  fig.add_subplot(312).imshow(img, cmap='gray', vmin=0, vmax=255)
  fig.add_subplot(313).imshow(img_seg, cmap='gray', vmin=0, vmax=255)
  fig.show()
  fig.savefig(f'{name}_seg.png', bbox_inches='tight', pad_inches=0)

  return img_seg, events_seg


### Execução

In [ ]:
img_seg_out, event_seg_out = main_seg_image('opencvpy.png', (100, 50), 1, 120)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('xmind.png', (1, 1), 1, 130)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('tree.png', (1, 1), 1, 130)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('buzz.png', (1, 1), 1, 150)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('sr.png', (500, 200), 1, 150)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('ball.png', (1, 1), 1, 74)

In [ ]:
img_seg_out, event_seg_out = main_seg_image('pica-pau.jpg', (1, 1), 1, 130)

### Animação

In [ ]:
mt_ani = [[0 for _ in range(img_seg_out.shape[1])]
          for _ in range(img_seg_out.shape[0])]

fig = plt.figure(figsize=(15, 15))

l = plt.imshow(mt_ani, cmap='gray', vmin=0, vmax=255)

def updatefig(k):
  line, col, v = k
  
  mt_ani[line][col] = v

  l.set_array(mt_ani)
  return [l]

ani = animation.FuncAnimation(fig, updatefig,
                              frames=list(filter(lambda x: x[2] > 200,  event_seg_out))[:1000],
                              interval=50, blit=True)
ani.save('Animação3.gif', writer = "pillow", fps=30 )  